***PART 1 - LOADING LIBRARIES AND DATABASE***

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import seaborn as sns

pd.options.display.max_columns = None #The dataset is huge. To see all of the columns, we change max columns
pd.options.display.max_seq_items=1000
pd.options.display.max_rows=1000

In [2]:
db_v2 ='https://drive.google.com/file/d/1XoqPujIOGHQStuAqM6Gzyqbkha4c5Y96/view?usp=sharing'
db_v3 ='https://drive.google.com/file/d/1W1V3_hl7yqbWyXF8w7mZSDTxCOx3GMzf/view?usp=sharing'
db_v4 ='https://drive.google.com/file/d/19ERs5bmAdxEfgUmTxgfIBhUoT6xPHzZy/view?usp=sharing'
db_v5 = 'https://drive.google.com/file/d/1X4YJP5fjfyk8f_TjSGBdIsTdv8MCCgY7/view?usp=sharing'
db_v8 = 'https://drive.google.com/file/d/1Ffbhy12m4JG9REEdSQwwewIFE0KUiEX3/view?usp=sharing'
db_v8bis = 'https://drive.google.com/file/d/13x5iZArdw03O4rBel4A6M2XrjoMpBcUJ/view?usp=sharing'
fname1 = db_v8
fname1='https://drive.google.com/uc?id=' + fname1.split('/')[-2]
dataset = pd.read_csv(fname1, low_memory=False)

In [3]:
dataset.shape

(6575, 62)

#MACHINE LEARNING



# Don't forget to run requirements.txt

command line : pip install -r ./requirements.txt

In [4]:
#kaggle ref that explain some of used models in this notbook. https://www.kaggle.com/code/serigne/stacked-regressions-top-4-on-leaderboard

#libraries 

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler,OneHotEncoder, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import r2_score, mean_squared_error,mean_squared_error,mean_squared_log_error,mean_absolute_error,explained_variance_score
import xgboost as xgb
import lightgbm as lgb

In [5]:
dataset.columns

Index(['Permit Number', 'Permit Type', 'Permit Type Definition',
       'Permit Creation Date', 'Block', 'Lot', 'Street Number', 'Street Name',
       'Street Suffix', 'Description', 'Current Status', 'Current Status Date',
       'Filed Date', 'Issued Date', 'Completed Date',
       'First Construction Document Date', 'Number of Proposed Stories',
       'Permit Expiration Date', 'Estimated Cost', 'Revised Cost',
       'Proposed Use', 'Proposed Units', 'Plansets',
       'Proposed Construction Type', 'Proposed Construction Type Description',
       'Site Permit', 'Supervisor District',
       'Neighborhoods - Analysis Boundaries', 'Zipcode', 'Location',
       'Record ID', 'address', 'lat', 'lon', 'Duration_construction_days',
       'Process_time_days', 'Year', 'Est_Cost_Infl', 'Rev_Cost_Infl',
       'Est_Cost_Infl_log10', 'Est_Cost_Infl_loge', 'lat_lon', 'address.1',
       'geometry', 'point', 'polygon', 'area_m2',
       'Number of Proposed Stories_', 'Number of Proposed Stories

In [6]:
dataset.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Street Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,Completed Date,First Construction Document Date,Number of Proposed Stories,Permit Expiration Date,Estimated Cost,Revised Cost,Proposed Use,Proposed Units,Plansets,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID,address,lat,lon,Duration_construction_days,Process_time_days,Year,Est_Cost_Infl,Rev_Cost_Infl,Est_Cost_Infl_log10,Est_Cost_Infl_loge,lat_lon,address.1,geometry,point,polygon,area_m2,Number of Proposed Stories_,Number of Proposed Stories_cat,Zipcode_,Proposed Construction Type_,Number of Proposed Stories_cat_f,Proposed Use_f,Proposed Construction Type_f,Number of Proposed Stories_bct,Number of Proposed Stories_cat_f_bct,Proposed Construction Type_f_bct,Proposed Units_bct,Proposed Use_f_bct,Duration_construction_days_bct,total_area_m2,cost_per_m2
0,9111378,2,new construction wood frame,1991-06-28,3566,037,3686,17th,St,erect a four story three unit residential bldg,complete,1993-09-20,1991-06-28,1992-11-30,1993-09-20,NaN,4.0,1994-11-30,250000.0,350000.0,apartments,3.0,2.0,5,wood frame (5),NaN,8.0,Castro/Upper Market,94114,POINT (-122.42798037718552 37.76316610254898),342335281878,3686 17th St,-122.427980,37.763166,294.0,521.0,1991,4.973752e+05,696325.256975,5.696684,13.117100,-4623.268159,"3686 17th St, San Francisco, CA 94114",NaN,POINT (-122.42798037718552 37.76316610254898),"POLYGON ((6004384.44156398 2106033.407080037, ...",147.915331,4.0,3-4 stories,94114,5,0,0,0,1.746189,0.0,0.0,1.486984,0.000000,7.659654,591.661324,840.641704
1,8707430,2,new construction wood frame,1987-05-29,1733,046,1254,18th,Av,NaN,complete,1988-10-07,1987-05-29,1988-01-20,1988-10-07,NaN,3.0,1990-01-20,394000.0,NaN,apartments,4.0,2.0,5,wood frame (5),NaN,4.0,Inner Sunset,94122,POINT (-122.47578567375196 37.76447248121622),161612112132,1254 18th Av,-122.475786,37.764472,261.0,236.0,1987,9.398079e+05,NaN,5.973039,13.753431,-4625.233438,"1254 18th Av, San Francisco, CA 94122","{""type"": ""Polygon"", ""coordinates"": [[[-122.476...",POINT (-122.47578567375196 37.76447248121622),"POLYGON ((5990594.490913103 2106871.242348627,...",253.922359,3.0,3-4 stories,94122,5,0,0,0,1.486984,0.0,0.0,1.746189,0.000000,7.451394,761.767077,1233.720846
2,7912377,2,new construction wood frame,1979-12-01,1361,064,3900,California,St,NaN,complete,1984-02-29,1979-12-01,1983-01-24,1984-02-29,NaN,3.0,1985-01-24,790000.0,NaN,apartments,8.0,2.0,5,wood frame (5),NaN,2.0,Inner Richmond,94118,POINT (-122.45952470081986 37.7858592269433),86903496726,3900 California St,-122.459525,37.785859,401.0,1150.0,1979,2.948572e+06,NaN,6.469612,14.896831,-4627.238361,"3900 California St, San Francisco, CA 94118","{""type"": ""Polygon"", ""coordinates"": [[[-122.459...",POINT (-122.45952470081986 37.7858592269433),POLYGON ((5995398.966155565 2114500.1454920806...,504.669823,3.0,3-4 stories,94118,5,0,0,0,1.486984,0.0,0.0,2.457309,0.000000,8.214724,1514.009469,1947.525221
3,8010466,2,new construction wood frame,1980-11-25,4159,074,1084,De Haro,St,NaN,complete,1985-07-05,1980-11-25,1984-08-06,1985-07-05,NaN,3.0,1985-08-06,210500.0,NaN,2 family dwelling,2.0,4.0,5,wood frame (5),NaN,10.0,Potrero Hill,94107,POINT (-122.40107686660524 37.7560791545319),1490171598,1084 De Haro St,-122.401077,37.756079,333.0,1350.0,1980,6.922231e+05,NaN,5.840246,13.447664,-4621.384747,"1084 De Haro St, San Francisco, CA 94107",NaN,POINT (-122.40107686660524 37.7560791545319),POLYGON ((6012157.402672291 2103366.9430118585...,167.842843,3.0,3-4 stories,94107,5,0,1,0,1.486984,0.0,0.0,1.161232,0.717735,7.880294,503.528529,1374.744586
4,8010466,2,new construction wood frame,1980-11-25,4159,073,1082,De Haro,St,NaN,complete,1985-07-05,1980-11-25,1984-08-06,1985-07-05,NaN,3.0,1985-08-06,210500.0,NaN,2 family dwelling,2.0,4.0,5,

In [7]:
# Separate target variable Y from features X
print("Separating labels from features...")
description_ML = "dataset V8 | default feature used + total_area_m2 + Neighborhoods | Hicham"#add why you do this experiment

features_list = [
    "Permit Type",
    "Number of Proposed Stories_",
    "Proposed Use",
    "Proposed Units",
    "Proposed Construction Type_", 
    'lat_lon',
    "Year",
    "Duration_construction_days",
    "Neighborhoods - Analysis Boundaries",
    "total_area_m2",
]

target_variable = "Est_Cost_Infl_log10"

X = dataset.loc[:,features_list]
Y = dataset.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
print(X.head())


Separating labels from features...
...Done.

Y : 
0    5.696684
1    5.973039
2    6.469612
3    5.840246
4    5.840246
Name: Est_Cost_Infl_log10, dtype: float64

X :
   Permit Type  Number of Proposed Stories_       Proposed Use  \
0            2                          4.0         apartments   
1            2                          3.0         apartments   
2            2                          3.0         apartments   
3            2                          3.0  2 family dwelling   
4            2                          3.0  2 family dwelling   

   Proposed Units Proposed Construction Type_      lat_lon  Year  \
0             3.0                           5 -4623.268159  1991   
1             4.0                           5 -4625.233438  1987   
2             8.0                           5 -4627.238361  1979   
3             2.0                           5 -4621.384747  1980   
4             2.0                           5 -4621.384747  1980   

   Duration_construction_da

In [8]:
# Automatically detect names of numeric/categorical columns
numeric_features = []
categorical_features = []
for i,t in X.dtypes.iteritems():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
    else :
        categorical_features.append(i)

print('Found numeric features ', numeric_features)
print('Found categorical features ', categorical_features)

Found numeric features  ['Permit Type', 'Number of Proposed Stories_', 'Proposed Units', 'lat_lon', 'Year', 'Duration_construction_days', 'total_area_m2']
Found categorical features  ['Proposed Use', 'Proposed Construction Type_', 'Neighborhoods - Analysis Boundaries']


In [9]:
#Manually selecting type of features
#numeric_features=['Number of Proposed Stories', 'Duration_construction_days','Year']
#categorical_features=['Permit Type', 'Proposed Construction Type', 'Neighborhoods - Analysis Boundaries', 'Proposed Use'] 

In [10]:
#Since we use Kfold, we don't divide data into train and test data set!
#for X_train and Y_train we use all dataset since we use K-fold
X_train = X
Y_train = Y 
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.0, random_state=0)#,stratify=Y)
print("...Done.")
print()

...Done.



In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6575 entries, 0 to 6574
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Permit Type                          6575 non-null   int64  
 1   Number of Proposed Stories_          6575 non-null   float64
 2   Proposed Use                         6575 non-null   object 
 3   Proposed Units                       6575 non-null   float64
 4   Proposed Construction Type_          6575 non-null   object 
 5   lat_lon                              6575 non-null   float64
 6   Year                                 6575 non-null   int64  
 7   Duration_construction_days           6574 non-null   float64
 8   Neighborhoods - Analysis Boundaries  6575 non-null   object 
 9   total_area_m2                        6575 non-null   float64
dtypes: float64(5), int64(2), object(3)
memory usage: 513.8+ KB


In [12]:
# Create pipeline for numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # missing values will be replaced by columns' mean
    ('scaler', StandardScaler())
])

In [13]:
# Create pipeline for categorical features
categorical_transformer = Pipeline(
    steps=[
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

In [14]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [15]:
"""
# Preprocessings on test set 
# IT IS COMMENTED SINCE WE USE K FOLD 
print("Performing preprocessings on test set...")
print(X_test.head()) 
X_test = preprocessor.transform(X_test) # Don't fit again !! The test set is used for validating decisions
# we made based on the training set, therefore we can only apply transformations that were parametered using the training set.
# Otherwise this creates what is called a leak from the test set which will introduce a bias in all your results.
print('...Done.')
print(X_test[0:5,:]) # MUST use this syntax because X_test is a numpy array and not a pandas DataFrame anymore
print()
"""

# Preprocessings on train set
print("Performing preprocessings on train set...")
#print(X.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X[0:5]) # MUST use this syntax because X_train is a numpy array and not a pandas DataFrame anymore
print()



Performing preprocessings on train set...
...Done.
   Permit Type  Number of Proposed Stories_       Proposed Use  \
0            2                          4.0         apartments   
1            2                          3.0         apartments   
2            2                          3.0         apartments   
3            2                          3.0  2 family dwelling   
4            2                          3.0  2 family dwelling   

   Proposed Units Proposed Construction Type_      lat_lon  Year  \
0             3.0                           5 -4623.268159  1991   
1             4.0                           5 -4625.233438  1987   
2             8.0                           5 -4627.238361  1979   
3             2.0                           5 -4621.384747  1980   
4             2.0                           5 -4621.384747  1980   

   Duration_construction_days Neighborhoods - Analysis Boundaries  \
0                       294.0                 Castro/Upper Market   
1    

#***---CROSS VALIDATION---***


We take the cross_val_score function of Sklearn. Then, we use the function Kfold where shuffle = True to allow a shuffle attribution as cross_val_score does not have this option.

***Define Metrics***





*** Extrait of python-sklearn manuel***

ref. https://scikit-learn.org/stable/modules/model_evaluation.html


***R² score, the coefficient of determination***

The r2_score function computes the coefficient of determination, usually denoted as

.

It represents the proportion of variance (of y) that has been explained by the independent variables in the model. It provides an indication of goodness of fit and therefore a measure of how well unseen samples are likely to be predicted by the model, through the proportion of explained variance.

As such variance is dataset dependent,
may not be meaningfully comparable across different datasets. Best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected (average) value of y, disregarding the input features, would get an score of 0.0.

***Mean squared error***

The mean_squared_error function computes mean square error, a risk metric corresponding to the expected value of the squared (quadratic) error or loss.


**Mean squared logarithmic error**

The mean_squared_log_error function computes a risk metric corresponding to the expected value of the squared logarithmic (quadratic) error or loss.

This metric is best to use when targets having exponential growth, such as population counts, average sales of a commodity over a span of years etc. Note that this metric penalizes an under-predicted estimate greater than an over-predicted estimate.

**Mean absolute percentage error***

The mean_absolute_percentage_error (MAPE), also known as mean absolute percentage deviation (MAPD), is an evaluation metric for regression problems. The idea of this metric is to be sensitive to relative errors. It is for example not changed by a global scaling of the target variable.


***Explained variance score***

The best possible score is 1.0, lower values are worse.

Link to R² score, the coefficient of determination: https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score

The difference between the explained variance score and the R² score, the coefficient of determination is that when the explained variance score does not account for systematic offset in the prediction. For this reason, the R² score, the coefficient of determination should be preferred in general.

In the particular case where the true target is constant, the Explained Variance score is not finite: it is either NaN (perfect predictions) or -Inf (imperfect predictions). Such non-finite scores may prevent correct model optimization such as grid-search cross-validation to be performed correctly. For this reason the default behaviour of explained_variance_score is to replace them with 1.0 (perfect predictions) or 0.0 (imperfect predictions). You can set the force_finite parameter to False to prevent this fix from happening and fallback on the original Explained Variance score.



In [16]:
#Validation function
n_folds = 4
random_state = 0

In [17]:
#R2

def R2_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=random_state).get_n_splits(X_train)
    R2= cross_val_score(model, X_train, Y_train, scoring="r2", cv = kf)
    return(R2)

In [18]:
#sqrt of negative mean squared error
def rmse_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=random_state).get_n_splits(X_train)
    rmse= np.sqrt(-cross_val_score(model, X_train, Y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [19]:
#sqrt of negative mean squred log error
def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=random_state).get_n_splits(X_train)
    rmsle= np.sqrt(-cross_val_score(model, X_train, Y_train, scoring="neg_mean_squared_log_error", cv = kf))
    return(rmsle)

In [20]:
#Mean absolute percentage error*
def mape_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=random_state).get_n_splits(X_train)
    mape= -cross_val_score(model, X_train, Y_train, scoring="neg_mean_absolute_percentage_error", cv = kf)
    return(mape)

In [21]:
#explained_variance_score 
def evs_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=random_state).get_n_splits(X_train)
    evs= cross_val_score(model, X_train, Y_train, scoring="explained_variance", cv = kf)
    return(evs)


***Definiton of base models***

***Linear Regression :***
    

Not that we dont apply Robustscaler()! It can be useful to make data more robust on outliers in the future. We can use sklearn's Robustscaler() method on pipeline


In [22]:
regressor0= LinearRegression()

***LASSO Regression :***
    



In [23]:
lasso = Lasso(alpha =0.0005, random_state=random_state)
#make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

***Elastic Net Regression :***




In [24]:
ENet = ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=random_state)
#make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

Kernel Ridge Regression :

In [25]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

***Gradient Boosting Regression :***


With huber loss, our model is more robust to outliers


In [26]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =random_state)

***XGBoost :***

In [27]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =random_state, nthread = -1)

**LightGBM - Gradient Boosting With LightGBM**

LightGBM is an algorithm developed by Microsoft in 2017. It has an advantage of changing the training algorithm that make the run time faster and likely result in a more effective model.

Ref. to the LightGBM algorithm paper:

    LightGBM: A Highly Efficient Gradient Boosting Decision Tree, 2017.
link : https://papers.nips.cc/paper/2017/hash/6449f44a102fde848669bdd9eb6b76fa-Abstract.html

In [28]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

***RandomForestRegressor :***    

In [29]:
randomForestRegressor = RandomForestRegressor(max_depth=16, min_samples_leaf =2, min_samples_split=8, n_estimators=140,random_state=random_state)


#Base models scores

Let's see how these base models perform on the data by evaluating the cross-validation metrics


In [30]:
models = [regressor0,
          lasso,
          ENet,
          #KRR,
          #GBoost, #there is an error. need to fix later. 
          #model_xgb,
          model_lgb,
          randomForestRegressor
          ]

model_names = ["\nLinear regressor model\n",
               "\nLasso model\n",
               "\nElastic Net regressor model\n",
               #"\nKernel Ridge model\n",
               #"\nGradient Boosting model\n",
               #"\nXGBoost model\n",
               "\nLGBM model\n",
               "\nRand Forest Regressor"
               ]


In [31]:
count = 0
for model in models :
  print(model_names[count])
  score_1 = R2_cv(model)
  print("R2 score - mean : {:.4f}  |  std : {:.4f}\n".format(score_1.mean(), score_1.std()))

  score_2 = rmse_cv(model)
  print("\nRoot mean squared error - mean : {:.4f}  |  std : {:.4f}\n".format(score_2.mean(), score_2.std()))

  score_3 = rmsle_cv(model)
  print("\nRMSLE -logarithmic error - mean : {:.4f}  |  std : {:.4f}\n".format(score_3.mean(), score_3.std()))

  score_4 = mape_cv(model)
  print("\nMean absolute percentage error - mean : {:.2f}  |  std : {:.2f}\n".format(score_4.mean(), score_4.std()))

  score_5 = evs_cv(model)
  print("\nExplained variance score - mean : {:.4f}  |  std : {:.4f}\n".format(score_5.mean(), score_5.std()))
  print("----------------END--------------------")
  count = count + 1
  execfile('../src/score_ML.py') #!python fichier.py ou %run fichier.py ne recuperrent pas des variables locales


Linear regressor model

R2 score - mean : 0.6869  |  std : 0.0804


Root mean squared error - mean : 0.2295  |  std : 0.0141


RMSLE -logarithmic error - mean : 0.0328  |  std : 0.0016


Mean absolute percentage error - mean : 0.03  |  std : 0.00


Explained variance score - mean : 0.7092  |  std : 0.0559

----------------END--------------------

Lasso model

R2 score - mean : 0.6868  |  std : 0.0730


Root mean squared error - mean : 0.2304  |  std : 0.0170


RMSLE -logarithmic error - mean : 0.0329  |  std : 0.0020


Mean absolute percentage error - mean : 0.03  |  std : 0.00


Explained variance score - mean : 0.7112  |  std : 0.0463

----------------END--------------------

Elastic Net regressor model

R2 score - mean : 0.6870  |  std : 0.0733


Root mean squared error - mean : 0.2303  |  std : 0.0168


RMSLE -logarithmic error - mean : 0.0329  |  std : 0.0019


Mean absolute percentage error - mean : 0.03  |  std : 0.00


Explained variance score - mean : 0.7112  |  std : 0.0468
